## Matching registry and pdms data

In [ ]:
import pandas as pd
import numpy as np
import getpass
import io
import msoffcrypto

In [ ]:
sah_sos_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/sos_sah_data/post_hoc_modified_aSAH_DATA_2009_2023_24122023.xlsx'
patient_id_link_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/pdms_data/Transfer Urs.pietsch@kssg.ch 22.01.24, 15_34/20240116_SAH_SOS_Patienten.csv'

In [ ]:
output_dir = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/pdms_data'

In [ ]:
password = getpass.getpass()

In [ ]:
decrypted_workbook = io.BytesIO()
with open(sah_sos_data_path, 'rb') as file:
    office_file = msoffcrypto.OfficeFile(file)
    office_file.load_key(password=password)
    office_file.decrypt(decrypted_workbook)

In [ ]:
registry_df = pd.read_excel(decrypted_workbook)
registry_df.head()

In [ ]:
patient_id_link_df = pd.read_csv(patient_id_link_data_path, sep=';', decimal='.')

In [ ]:
# Joining registry and pdms data
registry_df['Date_birth'] = pd.to_datetime(registry_df['Date_birth'], format='%d.%m.%Y')
registry_df['Date_admission'] = pd.to_datetime(registry_df['Date_admission'], format='%d.%m.%Y')
registry_df.rename(columns={'Name': 'JoinedName'}, inplace=True)

patient_id_link_df['GebDatum'] = pd.to_datetime(patient_id_link_df['GebDatum'], format='%Y-%m-%d')
patient_id_link_df['Eintritt'] = pd.to_datetime(patient_id_link_df['Eintritt'], format='%Y-%m-%d')

# join patient id link on registry by (first: dob, then admission date)
# joined_registry_df = registry_df.merge(patient_id_link_df, how='left', left_on=['Date_birth', 'Date_admission'],
#                                        right_on=['GebDatum', 'Eintritt'])
joined_registry_df = registry_df.merge(patient_id_link_df, how='left', left_on=['Date_birth'],
                                       right_on=['GebDatum'])
joined_registry_df['delta_admission'] = ((joined_registry_df['Date_admission'] - joined_registry_df['Eintritt']).dt.total_seconds() / (60 * 60 * 24)).abs()
# restrict to patients with admission date within 2 day of each other
joined_registry_df = joined_registry_df[(joined_registry_df['delta_admission'] <= 2) | (joined_registry_df['delta_admission'].isnull())]


In [ ]:
joined_registry_df[['JoinedName', 'Name', 'Vorname', 'Date_birth', 'Date_admission', 'GebDatum', 'Eintritt', 'delta_admission']]

In [ ]:
# check that joinedName contains Name if name is not null
joined_registry_df['name_verification'] = joined_registry_df.apply(
    lambda x: x['Name'] in x['JoinedName'] if pd.notnull(x['Name']) else True, axis=1)

In [ ]:
# Proceed with manual verification of the remaining patients with non matching names
joined_registry_df[joined_registry_df.name_verification == False][
    ['JoinedName', 'Name', 'Vorname', 'name_verification', 'Date_birth', 'Date_admission', 'GebDatum', 'Eintritt']]

##### try to find remaining patients where admission date does not match exactly

In [ ]:
# for all rows with nan in pNr, check if there is a patient with the same dob and admission year in the pdms data

for index, row in joined_registry_df.iterrows():
    if pd.notnull(row['pNr']):
        continue
    dob = row['Date_birth']
    year = row['Year']
    name = row['JoinedName']
    
    potential_match_df = patient_id_link_df[(patient_id_link_df['GebDatum'] == dob) & (patient_id_link_df['Eintritt'].dt.year == year)]
    
    if potential_match_df.shape[0] > 0 and potential_match_df['Name'].values[0] in name:
        joined_registry_df.at[index, 'pNr'] = potential_match_df['pNr'].values[0]
        print(f'Found match for {name} with pNr {row["pNr"]}')
    


## Remaining unmatched patients

patients in SOS db but not found in PDMS db

In [ ]:
joined_registry_df[(joined_registry_df['pNr'].isnull())]['Year'].value_counts()

In [ ]:
joined_registry_df[(joined_registry_df['pNr'].isnull()) & (joined_registry_df['Year'] >= 2019)][
    ['JoinedName', 'Name', 'Vorname', 'Date_birth', 'Date_admission', 'DCI_YN']]

### Save correspondence file

In [ ]:
registry_pdms_correspondence_df = joined_registry_df[
    ['SOS-CENTER-YEAR-NO.', 'JoinedName', 'Date_birth', 'pNr']]
registry_pdms_correspondence_df.rename(columns={'JoinedName.': 'sos_name'}, inplace=True)

In [ ]:
# registry_pdms_correspondence_df.to_csv(f'{output_dir}/registry_pdms_correspondence.csv', index=False)

In [ ]:
missing_patients_df = joined_registry_df[(joined_registry_df['pNr'].isnull())][
    ['SOS-CENTER-YEAR-NO.', 'JoinedName', 'Date_birth', 'Date_admission', 'Year', 'Date_discharge_ICU', 'DCI_YN']]
missing_patients_df.head()

In [ ]:
# save missing_patients_df
# allow for umlaute in encoding
missing_patients_df.to_csv(f'{output_dir}/missing_patients_in_pdms_data.csv', index=False, encoding='utf-8-sig')